<a href="https://colab.research.google.com/github/cgrundman/ml-stethoscope/blob/main/code/ml_stethoscope_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine-Learned Stethoscope - Modeling

*   List item
*   List item



## Import Statements

In [11]:
from google.colab import files, drive
import pandas as pd
import numpy as np

from scipy.io import wavfile
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import librosa as lb
import librosa.display as lbd
import os

## Load Data From Preprocessing

In [4]:
# files.upload() # Load processed_audio_files.zip, train.csv, and val.csv
!pip install -q PyDrive

In [13]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [7]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
https://drive.google.com/file/d/1VKVMgyBY4s8OE0HzTFD-mWMJliiy7_UZ/view?usp=share_link
https://drive.google.com/file/d/16tbj1KZbUXMKcxFd1JQIHsvNneEdJPqV/view?usp=sharing

https://drive.google.com/open?id=YOUR_FILE_ID

https://drive.google.com/file/d/1VKVMgyBY4s8OE0HzTFD-mWMJliiy7_UZ/view?usp=sharing

In [20]:
# download = drive.CreateFile({'id': '1VKVMgyBY4s8OE0HzTFD-mWMJliiy7_UZ'})
# download.GetContentFile('processed_audio_files')
! gdown 1VKVMgyBY4s8OE0HzTFD-mWMJliiy7_UZ

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1VKVMgyBY4s8OE0HzTFD-mWMJliiy7_UZ 



In [ ]:
!unzip -qq processed_audio_files.zip -d processed_audio_files

In [ ]:
files.upload() # Load train.csv

In [ ]:
files.upload() # Load val.csv

In [ ]:
train = pd.read_csv('content/train.csv')
val = pd.read_csv('content/val.csv')
train.head()

In [ ]:
ytrain = train.disease
yval = val.disease
yval

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
ytrain = le.fit_transform(ytrain)
yval = le.transform(yval)

## Feature Extraction

In [ ]:
def getFeatures(path):
    soundArr,sample_rate = lb.load(path)
    mfcc = lb.feature.mfcc(y=soundArr,sr=sample_rate)
    cstft = lb.feature.chroma_stft(y=soundArr,sr=sample_rate)
    mSpec = lb.feature.melspectrogram(y=soundArr,sr=sample_rate)

    return mfcc,cstft,mSpec

In [ ]:
root = '/content/processed_audio_files/'
mfcc, cstft, mSpec = [], [], []

for idx,row in val.iterrows():
    path = root + row['filename']
    a, b, c, d, e = getFeatures(path)
    mfcc.append(a)
    cstft.append(b)
    mSpec.append(c)
    
mfcc_val = np.array(mfcc)
cstft_val = np.array(cstft)
mSpec_val = np.array(mSpec)

In [ ]:
root='/content/processed_audio_files/'
mfcc, cstft, mSpec, tone, specCen = [], [], [], [], []

for idx,row in train.iterrows():
    path = root + row['filename']
    a, b, c, d, e = getFeatures(path)
    mfcc.append(a)
    cstft.append(b)
    mSpec.append(c)
    tone.append(d)
    specCen.append(e)
    
mfcc_train=np.array(mfcc)
cstft_train=np.array(cstft)
mSpec_train=np.array(mSpec)

## Testing Features

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0.00001,mode='min')
]

### a. MFCC Model

In [ ]:
mfcc_input = keras.layers.Input(shape=(20,259,1),name="mfccInput")
x = keras.layers.Conv2D(32,5,strides=(1,3),padding='same')(mfcc_input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
x = keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x = keras.layers.Conv2D(64,3,strides=(1,2),padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
x = keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x = keras.layers.Conv2D(96,2,padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
x = keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x = keras.layers.Conv2D(128,2,padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
mfcc_output = keras.layers.GlobalMaxPooling2D()(x)

mfcc_model = keras.Model(mfcc_input, mfcc_output, name="mfccModel")

In [ ]:
mfcc_model.summary()

### b. CROMA Model

In [ ]:
croma_input = keras.layers.Input(shape=(12,259,1),name="cromaInput")
x = keras.layers.Conv2D(32,5,strides=(1,3),padding='same')(croma_input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
x = keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x = keras.layers.Conv2D(64,3,strides=(1,2),padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
x = keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x = keras.layers.Conv2D(128,2,padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
croma_output = keras.layers.GlobalMaxPooling2D()(x)

croma_model = keras.Model(croma_input, croma_output, name="cromaModel")

In [ ]:
croma_model.summary()

### c. MSPEC model

In [ ]:
mSpec_input = keras.layers.Input(shape=(128,259,1),name="mSpecInput")
x = keras.layers.Conv2D(32,5,strides=(2,3),padding='same')(mSpec_input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
x = keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x = keras.layers.Conv2D(64,3,strides=(2,2),padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
x = keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x = keras.layers.Conv2D(96,2,padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
x = keras.layers.MaxPooling2D(pool_size=2,padding='valid')(x)

x = keras.layers.Conv2D(128,2,padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation(keras.activations.relu)(x)
mSpec_output = keras.layers.GlobalMaxPooling2D()(x)

mSpec_model = keras.Model(mSpec_input, mSpec_output, name="mSpecModel")

In [ ]:
mSpec_model.summary()

### d. Dense Layers

In [ ]:
input_mfcc = keras.layers.Input(shape=(20,259,1),name="mfcc")
mfcc = mfcc_model(input_mfcc)

input_croma = keras.layers.Input(shape=(12,259,1),name="croma")
croma = croma_model(input_croma)

input_mSpec = keras.layers.Input(shape=(128,259,1),name="mspec")
mSpec = mSpec_model(input_mSpec)


concat = keras.layers.concatenate([mfcc,croma,mSpec])
hidden = keras.layers.Dropout(0.2)(concat)
hidden = keras.layers.Dense(50,activation='relu')(concat)
hidden = keras.layers.Dropout(0.3)(hidden)
hidden = keras.layers.Dense(25,activation='relu')(hidden)
hidden = keras.layers.Dropout(0.3)(hidden)
output = keras.layers.Dense(8,activation='softmax')(hidden)

net = keras.Model([input_mfcc,input_croma,input_mSpec], output, name="Net")

In [ ]:
net.summary()

In [ ]:
# keras.utils.plot_model(net, "net.png", show_shapes=True)

In [ ]:
from keras import backend as K
net.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
K.set_value(net.optimizer.learning_rate, 0.001)

In [ ]:
history=net.fit(
    {"mfcc":mfcc_train,"croma":cstft_train,"mspec":mSpec_train},
    ytrain,
    validation_data=({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val},yval),
    epochs=100,verbose=0,
    callbacks=my_callbacks
)

In [ ]:
pd.DataFrame(history.history).plot()
plt.grid(True)
plt.gca().set_ylim(-0.1,1.1)
plt.show()

In [ ]:
net.evaluate({"mfcc":mfcc_val,"croma":cstft_val,"mspec":mSpec_val},yval)